In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers import wxgb

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    break
    
for i in ds_test.batch(60000):
    b = i
    break

In [2]:
xtrain = a[0].numpy().reshape([-1,28*28])
ytrain = a[1].numpy()

xtest = b[0].numpy().reshape([-1,28*28])
ytest = b[1].numpy()

dtrain = xgb.DMatrix(xtrain,label=ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 20,watchlist,evals_result=ed1_results)
    
    output = min(ed1_results['test']['merror'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['I']),
    'extra_dims': hp.choice('extra_dims',[0]),
    'objective': hp.choice('objective',['multi:softmax']),
    'eval_metric':hp.choice('eval_metric',[['merror']]),
    'num_class':hp.choice('num_class',[10]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.6461486911685859, 'colsample_bytree': 0.507762198600348, 'eta': 0.02894981656189124, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 0, 'lambda': 0.00039417698396286875, 'max_depth': 3, 'min_child_weight': 0.020806279942577326, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.6824255351732353}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[17:58:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open

[0]	train-merror:0.41568	test-merror:0.41290                           

[1]	train-merror:0.32065	test-merror:0.31990                           

[2]	train-merror:0.25877	test-merror:0.25420                           

[3]	train-merror:0.21947	test-merror:0.21270                           

[4]	train-merror:0.19078	test-merror:0.18790                           

[5]	train-merror:0.17717	test-merror:0.17120                           

[6]	train-merror:0.16105	test-merror:0.15530                           

[7]	train-merror:0.14928	test-merror:0.14530                           

[8]	train-merror:0.13912	test-merror:0.13320                           

[9]	train-merror:0.12952	test-merror:0.12660                           

[10]	train-merror:0.12185	test-merror:0.11880                          

[11]	train-merror:0.11465	test-merror:0.11150                          

[12]	train-merror:0.10975	test-merror:0.10740                          

[13]	train-merror:0.10550	test-merror:0.10330      

[5]	train-merror:0.05428	test-merror:0.07150                           

[6]	train-merror:0.05323	test-merror:0.07080                           

[7]	train-merror:0.05263	test-merror:0.06940                           

[8]	train-merror:0.05208	test-merror:0.06670                           

[9]	train-merror:0.05193	test-merror:0.06750                           

[10]	train-merror:0.05068	test-merror:0.06590                          

[11]	train-merror:0.04827	test-merror:0.06480                          

[12]	train-merror:0.04783	test-merror:0.06400                          

[13]	train-merror:0.04743	test-merror:0.06410                          

[14]	train-merror:0.04775	test-merror:0.06330                          

[15]	train-merror:0.04722	test-merror:0.06390                          

[16]	train-merror:0.04627	test-merror:0.06320                          

[17]	train-merror:0.04587	test-merror:0.06310                          

[18]	train-merror:0.04573	test-merror:0.06260      

[9]	train-merror:0.32768	test-merror:0.32300                            

[10]	train-merror:0.32313	test-merror:0.31950                           

[11]	train-merror:0.32283	test-merror:0.31990                           

[12]	train-merror:0.32155	test-merror:0.31740                           

[13]	train-merror:0.32217	test-merror:0.31840                           

[14]	train-merror:0.31885	test-merror:0.31530                           

[15]	train-merror:0.31568	test-merror:0.31220                           

[16]	train-merror:0.31720	test-merror:0.31370                           

[17]	train-merror:0.31390	test-merror:0.30850                           

[18]	train-merror:0.31215	test-merror:0.30770                           

[19]	train-merror:0.31235	test-merror:0.30720                           

{'alpha': 1.068036664197713, 'btype': 'I', 'colsample_bylevel': 0.9172694819039757, 'colsample_bytree': 0.8240888288434373, 'eta': 0.6726035725856117, 'eval_metric': ('merror',), 'extra_

[12]	train-merror:0.10213	test-merror:0.10650                            

[13]	train-merror:0.10085	test-merror:0.10420                            

[14]	train-merror:0.10015	test-merror:0.10400                            

[15]	train-merror:0.09880	test-merror:0.10280                            

[16]	train-merror:0.09688	test-merror:0.10120                            

[17]	train-merror:0.09623	test-merror:0.10030                            

[18]	train-merror:0.09522	test-merror:0.09980                            

[19]	train-merror:0.09427	test-merror:0.09920                            

{'alpha': 2.6512624992325315e-05, 'btype': 'I', 'colsample_bylevel': 0.8887715695077836, 'colsample_bytree': 0.9604505033121815, 'eta': 0.44729291365920454, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 0.008626081909695228, 'lambda': 0, 'max_depth': 3, 'min_child_weight': 0.028601818828287762, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.7738016092503124}
Overwriting para

...

[19]	train-merror:0.07872	test-merror:0.08980                            

{'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.939147387058243, 'colsample_bytree': 0.979947631067565, 'eta': 0.7428678306512836, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 0.006700893319314933, 'lambda': 0, 'max_depth': 7, 'min_child_weight': 0.005635413316632553, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.6984055396623483}
Overwriting param `num_class`                                            
Overwriting param `objective` while setting `obj` in train.              
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                  
Setting param `disable_default_eval_metric` to 1.                        
[20:56:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used

Overwriting param `num_class`                                            
Overwriting param `objective` while setting `obj` in train.              
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                  
Setting param `disable_default_eval_metric` to 1.                        
[21:09:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.27160	test-merror:0.26850                             

[1]	train-merror:0.20818	test-merror:0.20150                             

[2]	train-merror:0.16532	test-merror:0.16020                             

[3]	train-merror:0.14915	test-merr

Setting param `disable_default_eval_metric` to 1.                        
[21:21:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.09915	test-merror:0.12710                             

[1]	train-merror:0.05498	test-merror:0.08670                             

[2]	train-merror:0.04052	test-merror:0.07260                             

[3]	train-merror:0.03195	test-merror:0.06560                             

[4]	train-merror:0.02522	test-merror:0.06060                             

[5]	train-merror:0.02020	test-merror:0.05500                             

[6]	train-merror:0.01640	test-merror:0.04950                             

[7]	train-merror:0.01307	test-merror:0.04680  

[0]	train-merror:0.13463	test-merror:0.14240                             

[1]	train-merror:0.09390	test-merror:0.10180                             

[2]	train-merror:0.07635	test-merror:0.08250                             

[3]	train-merror:0.06565	test-merror:0.07500                             

[4]	train-merror:0.06012	test-merror:0.06690                             

[5]	train-merror:0.05500	test-merror:0.06430                             

[6]	train-merror:0.05112	test-merror:0.06190                             

[7]	train-merror:0.04735	test-merror:0.05910                             

[8]	train-merror:0.04385	test-merror:0.05530                             

[9]	train-merror:0.04162	test-merror:0.05450                             

[10]	train-merror:0.03917	test-merror:0.05240                            

[11]	train-merror:0.03733	test-merror:0.05120                            

[12]	train-merror:0.03553	test-merror:0.05030                            

[13]	train-merror:0.03363

[5]	train-merror:0.20148	test-merror:0.19990                             

[6]	train-merror:0.19038	test-merror:0.18870                             

[7]	train-merror:0.17828	test-merror:0.17290                             

[8]	train-merror:0.16972	test-merror:0.16500                             

[9]	train-merror:0.16193	test-merror:0.15820                             

[10]	train-merror:0.15373	test-merror:0.15100                            

[11]	train-merror:0.14663	test-merror:0.14640                            

[12]	train-merror:0.14162	test-merror:0.14070                            

[13]	train-merror:0.13613	test-merror:0.13570                            

[14]	train-merror:0.13128	test-merror:0.13270                            

[15]	train-merror:0.12687	test-merror:0.12750                            

[16]	train-merror:0.12363	test-merror:0.12360                            

[17]	train-merror:0.11895	test-merror:0.12130                            

[18]	train-merror:0.11527

In [4]:
print(best_val)
print(space_eval(spc, best))

0.0306
{'alpha': 0.22960412517893544, 'btype': 'I', 'colsample_bylevel': 0.9992802458132009, 'colsample_bytree': 0.9871650956900173, 'eta': 0.3738600803261662, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 0.0005189482695228505, 'lambda': 0.17652268034672527, 'max_depth': 9, 'min_child_weight': 8.669337368188545e-05, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.8860890420514498}


In [13]:
from sklearn.neural_network import MLPClassifier

#mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, alpha=1e-4,
#                    solver='sgd', verbose=10, random_state=1,
#                    learning_rate_init=.1)
ms_params = {
    'hidden_layer_sizes' : [[50],
                            [100],
                            [50,50],
                            [100,100],
                            [200,200],
                            [200,200,200]
                           ],
    'max_iter':[400]
}
ms = GridSearchCV(MLPClassifier(), ms_params)
ms.fit(xtrain,y=ytrain)

NameError: name 'GridSearchCV' is not defined

In [ ]:
print(ms.cv_results_['params'][np.argmax(ms.cv_results_['mean_test_score'])])
max(ms.cv_results_['mean_test_score'])

In [ ]:
from sklearn.model_selection import GridSearchCV

gs_params = {
    'btype':['R','I','Rn','In'],
    'extra_dims':[0,64,128,256],
    'max_depth':[4],
    'eta':[0.1],
    'objective':['multi:softmax'],
    'subsample':[0.5],
    'colsample_bytree':[0.5],
    'n_estimators':[10],
    'min_child_weight':[1],
    'eval_metric':[['merror']],
    'num_class':[10]
}

gs = GridSearchCV(wxgbModel(), gs_params)
gs.fit(xtrain, y=ytrain)

In [ ]:
gs.cv_results_

In [ ]:
from wideboost.wrappers import wxgb

param = {'max_depth':2, 'objective':'multi:softmax','eval_metric':['merror'], 'num_class':10 }
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['eta'] = 0.001
param['btype'] = 'Rn'
param['extra_dims'] = 1024
param['lambda'] = 6
num_round = 200
#dtrain = xgb.DMatrix(xtrain,label=ytrain)
watchlist = [(dtrain,'train'),(dtest,'test')]
results = dict()
bst = wxgb.train(param, dtrain, num_round,watchlist,evals_result=results)



In [ ]:
plt.plot(results['train']['merror'])
plt.plot(results['test']['merror'])
plt.show()